In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("neo.csv",index_col=False)
df = df.drop(["id","name","orbiting_body","sentry_object"],axis=1)
df["hazardous"] = np.where(df["hazardous"] == False, 0, 1)
x = df.drop(["hazardous"],axis=1)
y= df["hazardous"]

In [3]:
x=StandardScaler().fit_transform(x)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=20, train_size = 0.80)

In [5]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [6]:
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [7]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import TensorBoard as tb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
tbCallbacks = tb(log_dir="logs/", histogram_freq=1)
EScallback =EarlyStopping(monitor='loss', patience=3)

In [10]:
model = Sequential([
    Dense(units=10,activation='relu',input_shape=(X_train.shape[1:])),
    # Dense(units=10,activation='sigmoid'),
    Dense(units=20,activation='relu'),
    Dense(units=40,activation='sigmoid'),
    Dense(units=30,activation='relu'),
    Dense(units=15,activation='sigmoid'),
    # Dense(units=20,activation='relu'),
    # Dense(units=5,activation='relu'),
    Dense(units=1,activation='linear')
    ])
model.compile(optimizer=Adam(learning_rate=1e-2),loss=BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 10)                60        
                                                                 
 dense_8 (Dense)             (None, 20)                220       
                                                                 
 dense_9 (Dense)             (None, 40)                840       
                                                                 
 dense_10 (Dense)            (None, 30)                1230      
                                                                 
 dense_11 (Dense)            (None, 15)                465       
                                                                 
 dense_12 (Dense)            (None, 1)                 16        
                                                                 
Total params: 2,831
Trainable params: 2,831
Non-traina

In [11]:
hist= model.fit(X_train,y_train ,epochs=100,    validation_split=0.2, verbose=2 ,callbacks= [tbCallbacks,EScallback])

Epoch 1/100
1817/1817 - 2s - loss: 0.2099 - accuracy: 0.9015 - val_loss: 0.2004 - val_accuracy: 0.9060 - 2s/epoch - 1ms/step
Epoch 2/100
1817/1817 - 1s - loss: 0.1979 - accuracy: 0.9017 - val_loss: 0.1878 - val_accuracy: 0.9060 - 1s/epoch - 753us/step
Epoch 3/100
1817/1817 - 1s - loss: 0.1930 - accuracy: 0.9065 - val_loss: 0.1907 - val_accuracy: 0.9144 - 1s/epoch - 747us/step
Epoch 4/100
1817/1817 - 1s - loss: 0.1920 - accuracy: 0.9089 - val_loss: 0.1837 - val_accuracy: 0.9133 - 1s/epoch - 747us/step
Epoch 5/100
1817/1817 - 1s - loss: 0.1917 - accuracy: 0.9086 - val_loss: 0.1833 - val_accuracy: 0.9145 - 1s/epoch - 747us/step
Epoch 6/100
1817/1817 - 1s - loss: 0.1915 - accuracy: 0.9085 - val_loss: 0.1865 - val_accuracy: 0.9150 - 1s/epoch - 757us/step
Epoch 7/100
1817/1817 - 1s - loss: 0.1926 - accuracy: 0.9092 - val_loss: 0.1843 - val_accuracy: 0.9152 - 1s/epoch - 747us/step
Epoch 8/100
1817/1817 - 1s - loss: 0.1913 - accuracy: 0.9086 - val_loss: 0.1853 - val_accuracy: 0.9153 - 1s/epoch

In [12]:
logits = model(X_test)
yhat = tf.nn.sigmoid(logits)

In [13]:
y_test = np.reshape(y_test,(y_test.shape[0],1))
y_test.shape

(18168, 1)

In [14]:
yhat=yhat.numpy()
yhat.shape

(18168, 1)

In [15]:
o,z=0,0
yhatN=[]
for i in range(0,yhat.shape[0]):
    if(yhat[i][0] < 0.5):
        yhatN.append(0)
        z=z+1
    else:
        yhatN.append(1)
        o=o+1
print(o,z)  

417 17751


In [16]:
from sklearn.metrics import f1_score
f1_score(y_test, yhatN, average='micro')

0.9149603698811096

In [17]:
from sklearn.metrics import classification_report
target_name=['false','true']
print(classification_report(y_test, yhatN, target_names=target_name))

              precision    recall  f1-score   support

       false       0.92      0.99      0.95     16418
        true       0.75      0.18      0.29      1750

    accuracy                           0.91     18168
   macro avg       0.83      0.59      0.62     18168
weighted avg       0.90      0.91      0.89     18168



In [20]:
model.save('NN_weights')

INFO:tensorflow:Assets written to: NN_weights\assets


In [22]:
reconstructed_model = tf.keras.models.load_model("NN_weights")

In [24]:
logits = reconstructed_model(X_test)
yhat = tf.nn.sigmoid(logits)